In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np

import reference_LFs as refLF
from paus_utils import *

from jpasLAEs.utils import bin_centers

import pickle

In [ ]:
def load_combined_LF(survey_list, nb1, nb2):
    this_hist = None
    for survey_name in survey_list:
        LF_name = f'Lya_LF_nb{nb1}-{nb2}_{survey_name}'
        pathname = f'/home/alberto/almacen/PAUS_data/Lya_LFs/{LF_name}'
        filename_hist = f'{pathname}/hist_i_mat_{survey_name}.npy'
        hist_i_mat = np.load(filename_hist)

        L_bins = np.load(f'{pathname}/LF_L_bins.npy')
        bin_width = [L_bins[i + 1] - L_bins[i] for i in range(len(L_bins) - 1)]

        if this_hist is None:
            this_hist = hist_i_mat
        else:
            this_hist += hist_i_mat


    L_LF_err_percentiles = np.percentile(this_hist, [16, 50, 84], axis=0)
    hist_median = L_LF_err_percentiles[1]

    # yerr_minus = np.load(f'{pathname}/LF_err_minus.npy')
    # yerr_plus = np.load(f'{pathname}/LF_err_plus.npy')

    eff_vol = Lya_effective_volume(nb1, nb2, survey_name)

    this_LF = hist_median / bin_width / eff_vol
    # LF_boots = np.load(f'{pathname}/median_LF_boots.npy')
    # # Fix yerr_minus when LF_boots == 0
    # yerr_minus[LF_boots == 0] = this_LF[LF_boots == 0]

    this_LF_dict = {
        'LF_bins': bin_centers(L_bins),
        'LF_total': this_LF,
        # 'LF_total_err': [yerr_minus, yerr_plus],
    }

    return this_LF_dict

In [ ]:
survey_list = ['foo']

nb1, nb2 = 1, 10

LyaLF = load_combined_LF(survey_list, nb1, nb2)

fig, ax = plt.subplots(figsize=(6, 4))

ax.errorbar(LyaLF['LF_bins'], LyaLF['LF_total'],
            yerr=None,
            fmt='s', ls='', mfc='limegreen', mec='k',
            ms=7)

ax.set(yscale='log',
       xlim=(42.5, 46), ylim=(1e-8, 1e-4))

plt.show()